In [53]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA


In [60]:
import pandas as pd

In [54]:
import requests
from bs4 import BeautifulSoup

s = 'BIDU stock'
url = "http://www.google.com/search?q="+s+"&tbm=nws"
r1 = requests.get(url)
coverpage = r1.content
# print(coverpage)
soup1 = BeautifulSoup(coverpage, 'html5lib')
headlines = soup1.find_all('div', class_='BNeawe vvjwJb AP7Wnd')
headlines

[<div class="BNeawe vvjwJb AP7Wnd">Could Google Suffer the Same Fate as Baidu?</div>,
 <div class="BNeawe vvjwJb AP7Wnd">Why Baidu's Stock Rose 16.4% in November</div>,
 <div class="BNeawe vvjwJb AP7Wnd">What Is Baidu, Inc.'s (NASDAQ:BIDU) Share Price Doing?</div>,
 <div class="BNeawe vvjwJb AP7Wnd">Is Baidu, Inc. (BIDU) A Good Stock To Buy?</div>,
 <div class="BNeawe vvjwJb AP7Wnd">Stock in the NEWS Limelight:: Baidu, Inc., (NASDAQ: BIDU)</div>,
 <div class="BNeawe vvjwJb AP7Wnd">3 Ways Baidu Plans to Make a Comeback in China</div>,
 <div class="BNeawe vvjwJb AP7Wnd">Baidu, Inc. [BIDU] – Is there any real value to BIDU Stock or is it vaporware?</div>,
 <div class="BNeawe vvjwJb AP7Wnd">The Most Disappointing Tech Stock of 2019</div>,
 <div class="BNeawe vvjwJb AP7Wnd">Stocks This Week: Buy Tesla And Sell Short Baidu</div>,
 <div class="BNeawe vvjwJb AP7Wnd">Nvidia Shares Hit Two-Year Highs After Unveiling Chip Deals With Alibaba, Baidu and Didi</div>]

In [58]:
sia = SIA()

In [59]:
import csv
import pandas as pd

# stock market lexicon
stock_lex = pd.read_csv('lexicon_data/stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

# Loughran and McDonald
positive = []
with open('lexicon_data/lm_positive.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        positive.append(row[0].strip())
    
negative = []
with open('lexicon_data/lm_negative.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        entry = row[0].strip().split(" ")
        if len(entry) > 1:
            negative.extend(entry)
        else:
            negative.append(entry[0])

final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [62]:

results = []

for line in headlines:
    casted_line = str(line)[34:-6]
    pol_score = sia.polarity_scores(casted_line)
    pol_score['headline'] = casted_line
    results.append(pol_score)

In [63]:
results = pd.DataFrame(results)

In [64]:
results

,compound,headline,neg,neu,pos
0,-0.5550,Could Google Suffer the Same Fate as Baidu?,0.712,0.185,0.103
1,0.1625,Why Baidu's Stock Rose 16.4% in November,0.139,0.512,0.349
2,0.3732,"What Is Baidu, Inc.'s (NASDAQ:BIDU) Share Pric...",0.000,0.523,0.477
3,0.6143,"Is Baidu, Inc. (BIDU) A Good Stock To Buy?",0.092,0.272,0.637
4,0.3697,"Stock in the NEWS Limelight:: Baidu, Inc., (NA...",0.197,0.464,0.340
5,0.3905,3 Ways Baidu Plans to Make a Comeback in China,0.213,0.305,0.482
6,0.5353,"Baidu, Inc. [BIDU] – Is there any real value t...",0.175,0.341,0.485
7,-0.3905,The Most Disappointing Tech Stock of 2019,0.533,0.203,0.265
8,-0.0856,Stocks This Week: Buy Tesla And Sell Short Baidu,0.201,0.269,0.530
9,0.4615,Nvidia Shares Hit Two-Year Highs After Unveili...,0.133,0.489,0.378
